This notebook performs the experiment, where a DNN is the base model to mitigate. We consider two fairness definitions: statistical parity and equal opportunity. All three compared methods are assessed on statistical parity, while only FairWhere and PROMIS are assessed on equal opportunity. For FairWhere we define the distance for each partition as the absoluted difference of the partition recall and the overall model recall and for statistical parity we use the positive ratio as metric
The three types of audit regions are tested and with the 2 fairness notions result in 6 experiments.

Initially the base DNN is trained. FairWhere continues training on the training set with the goal of mitigating spatial bias. At the start of fairness training, each sampled partitioning is used for 5 epochs before transitioning to the next, iterating over 120 different samples (equivalent to five full enumerations over all 24 partitioning candidates). Subsequently, each epoch samples a new partitioning, continuing for 720 samples (equivalent to thirty full enumerations). In total, each partitioning is expected to be used for 50 epochs throughout
the training process.

The predictions of the base and corrected (by FairWhere) model are saved for later analysis.

The maximum budget, given as input for PROMIS and SpatialFlips is computed by the absolute difference of the predictions on the validation set between the base DNN and FairWhere models. 

PROMIS methods apply the thresholds adjustment approach using the validation set, while SpatialFlips apply flips directly on the test set.

The "pretrained" PROMIS, SpatialFlip models are saved for later analysis on the test set.

You may opt which experiment to run by uncommenting the relate part in the section choose experiment to run. To rerun an experiment restart the notebook (for reproducibility).

For each experiment the directory dnn_exp/regions_\<partitioning_name>|pred_dnn_crime is created.

Inside the directory the following files are created:

* for FairWhere:
    * keras_models/\<fairness_notion>_crime_partition_model.keras
    * crime_\<fairness_notion>_where_fit_time.txt
    * crime_\<fairness_notion>\_where_model_\<set>_pred.csv
* for SpatialFlip:
    * spatial_flip_models/\<fairness_notion>/iter.pkl
* for PROMIS:
    * spatial_optim_models/\<fairness_notion>/<promis_method_name>.pkl (for promis_opt method the wlimit_\<work_limit> is appended)

In [1]:
import pandas as pd
import numpy as np
import random
from sklearn.metrics import recall_score,f1_score
from tensorflow.keras.models import Model
from tensorflow.keras import Sequential
import tensorflow as tf

import numpy as np
import random
import time
import matplotlib.pyplot as plt
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense, BatchNormalization, Dropout
from tensorflow.keras.models import load_model
from tensorflow.keras import backend as K

from tqdm.notebook import tqdm
import os
import sys

sys.path.append(os.path.abspath(os.path.join('..')))

from methods.models.optimization_model import SpatialOptimFairnessModel 
from methods.models.spatial_flip_model import SpatialFlipFairnessModel
from utils.results_names_utils import combine_world_info
from utils.data_utils import get_y, read_scanned_regs, get_pos_info_regions
import ast

## Choose Experiment to Run

### Equal Opportunity

Audit Regions = Grids

In [ ]:
# fair_notion = "equal_opportunity" 
# partioning_type_name = "5_x_5"
# n_flips_start = 400
# overlap=True

Audit Regions = Clusters

In [2]:
# fair_notion = "equal_opportunity" 
# partioning_type_name = "non_overlap_k_8"
# n_flips_start = 400
# overlap=False

Audit Regions = Scan Regions

In [ ]:
# fair_notion = "equal_opportunity" 
# partioning_type_name = "overlap_k_10_radii_4"
# n_flips_start = 1000
# overlap=True

### Statistical Parity

Audit Regions = Grids

In [ ]:
# fair_notion = "statistical_parity" 
# partioning_type_name = "5_x_5"
# n_flips_start = 1200
# overlap=True

Audit Regions = Clusters

In [3]:
# fair_notion = "statistical_parity" 
# partioning_type_name = "non_overlap_k_8"
# n_flips_start = 900
# overlap=False

Audit Regions = Scan Regions

In [ ]:
fair_notion = "statistical_parity" 
partioning_type_name = "overlap_k_10_radii_4"
n_flips_start = 3000
overlap=True

In [ ]:
data_path = "../../data/"
prediction_dir = f'{data_path}predictions/' # folder in data path to save the base model predictions
partitioning_dir = f'{data_path}partitionings/' # folder in data path to save the partitionings
preprocess_dir = f'{data_path}preprocess/'
dataset_name = "crime"
clf_name = "dnn"
results_path = "../../results/dnn_exp/"

In [ ]:
res_desc_label, partioning_name, prediction_name = combine_world_info(
    dataset_name, partioning_type_name, clf_name
)
results_path = f"{results_path}{res_desc_label}/"
base_model_fname = f"{data_path}clf/dnn_{dataset_name}.keras"
keras_models_dir = f'{results_path}keras_models/{fair_notion}/' # folder to save base and where model
sp_opt_models_dir = f'{results_path}spatial_optim_models/{fair_notion}/'
for dir in [results_path, keras_models_dir]:
    os.makedirs(dir, exist_ok=True)

In [ ]:
SCORE_LABEL = 'recall' if fair_notion == 'equal_opportunity' else 'pr'
print(f"SCORE_LABEL: {SCORE_LABEL}")


In [ ]:
def dice(y_true, y_pred):
    y_true = tf.cast(y_true, tf.float32)
    y_pred = tf.cast(y_pred, tf.float32)
    intersect = tf.reduce_sum(y_pred * y_true, axis=0) + K.epsilon()
    denominator = tf.reduce_sum(y_pred, axis=0) + tf.reduce_sum(y_true, axis=0)
    dice_scores = 2 * intersect / (denominator + tf.keras.backend.epsilon())
    return 1 - dice_scores

def custom_loss(y_true, y_pred):
    loss = dice(y_true, y_pred)
    return loss

BATCH_SIZE = 64*64
LEARNING_RATE = 0.0001
LEARNING_RATE_INIT = LEARNING_RATE
EPOCH_TRAIN = 100

SEED = 42
tf.random.set_seed(SEED)
np.random.seed(SEED)
random.seed(SEED)

## Read Data

In [9]:
# Load data
X_train = pd.read_csv(f"{preprocess_dir}X_train_crime.csv")
X_val = pd.read_csv(f"{preprocess_dir}X_val_crime.csv")
X_test = pd.read_csv(f"{preprocess_dir}X_test_crime.csv")
y_train = pd.read_csv(f"{preprocess_dir}y_train_crime.csv")
y_val = pd.read_csv(f"{preprocess_dir}y_val_crime.csv")
y_test = pd.read_csv(f"{preprocess_dir}y_test_crime.csv")

##  Train Base Model

In [ ]:
initializer = tf.keras.initializers.TruncatedNormal(stddev=0.5, seed=SEED)
base_model = Sequential([
    Dense(100, activation='elu', input_dim=X_train.shape[1], kernel_initializer=initializer),
    BatchNormalization(),
    Dropout(0.1),
    Dense(50, activation='elu', kernel_initializer=initializer),
    BatchNormalization(),
    Dropout(0.1),
    Dense(2, activation='softmax', kernel_initializer=initializer),
])

print(base_model.summary())


optimizer = Adam(learning_rate=LEARNING_RATE)
base_model.compile(optimizer=optimizer, loss=custom_loss, metrics=['accuracy'])

# Start Training
history = base_model.fit(
    X_train,
    y_train,
    batch_size=BATCH_SIZE,
    epochs=EPOCH_TRAIN,
    shuffle=True,
    validation_data=(X_val, y_val),
)

plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Val Accuracy')
plt.legend()
plt.show()

plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.legend()
plt.show()

In [ ]:
#Predict the label
y_train_pred_base = base_model.predict(X_train, batch_size=BATCH_SIZE)
y_val_pred_base =  base_model.predict(X_val, batch_size=BATCH_SIZE)
y_test_pred_base =  base_model.predict(X_test, batch_size=BATCH_SIZE)

#Change Softmax to class
class_y_train_pred_base = np.argmax(y_train_pred_base, axis=-1)
class_y_val_pred_base = np.argmax(y_val_pred_base, axis=-1)
class_y_test_pred_base = np.argmax(y_test_pred_base, axis=-1)

#Prediction Value Count
print(np.unique(class_y_train_pred_base,return_counts=True))
print(np.unique(class_y_val_pred_base,return_counts=True))
print(np.unique(class_y_test_pred_base,return_counts=True))

base_test_f1 = f1_score(y_test,class_y_test_pred_base)
print(f"F1 train: {f1_score(y_train,class_y_train_pred_base)}, F1 val: {f1_score(y_val,class_y_val_pred_base)}, F1 test: , {f1_score(y_test,class_y_test_pred_base)}")

## Save Base Model Predictions and Generated Probabilities

In [ ]:
pos_class_prob_idx = 1  # Positive class index

y_train_base_pred_proba = base_model.predict(X_train, batch_size=BATCH_SIZE)[:, pos_class_prob_idx]
y_val_base_pred_proba = base_model.predict(X_val, batch_size=BATCH_SIZE)[:, pos_class_prob_idx]
y_test_base_pred_proba = base_model.predict(X_test, batch_size=BATCH_SIZE)[:, pos_class_prob_idx]

preds_df = pd.DataFrame({
    "pred": class_y_train_pred_base,
    "pred_proba": y_train_base_pred_proba,
})

val_preds_df = pd.DataFrame({
    "pred": class_y_val_pred_base,
    "prob": y_val_base_pred_proba
})

test_preds_df = pd.DataFrame({
    "pred": class_y_test_pred_base,
    "prob": y_test_base_pred_proba
})

preds_df.to_csv(f"{prediction_dir}train_{prediction_name}.csv", index=False)
val_preds_df.to_csv(f"{prediction_dir}val_{prediction_name}.csv", index=False)
test_preds_df.to_csv(f"{prediction_dir}test_{prediction_name}.csv", index=False)

## Apply Optimization Correction Methods

### Read Data

In [ ]:
# read data for other spatial bias mitigation models
val_pred_base_df = pd.read_csv(f"{prediction_dir}val_{prediction_name}.csv")
val_pred_partition_df = pd.read_csv(
    f"{results_path}{dataset_name}_{fair_notion}_where_model_val_pred.csv"
)
y_val_df = pd.read_csv(f"{preprocess_dir}y_val_{dataset_name}.csv")
y_val = get_y(y_val_df, "label")
pts_per_region_val_df = read_scanned_regs(
    f"{partitioning_dir}val_{partioning_name}.csv"
)
pts_per_region_val = pts_per_region_val_df["points"].tolist()
y_val_pred_base = get_y(val_pred_base_df, "pred")
y_val_base_pred_proba = get_y(val_pred_base_df, "prob")
y_val_pred_partition = get_y(val_pred_partition_df, "pred")

### Compute the Maximum Budget

In [ ]:
if fair_notion == "equal_opportunity":
    # Get positive class information to consider equal opportunity
    pos_val_y_true_indices, _ = get_pos_info_regions(y_val, pts_per_region_val)
    pos_val_y_pred_base = y_val_pred_base[pos_val_y_true_indices]
    pos_y_val_pred_partition = y_val_pred_partition[pos_val_y_true_indices]

# Calculate the differences between the base and partition model
# to use it as budget for the mitigation models
base_partition_diffs = (
    np.sum(np.abs(y_val_pred_base - y_val_pred_partition))
    if fair_notion == "statistical_parity"
    else np.sum((np.abs(pos_val_y_pred_base - pos_y_val_pred_partition)))
)
print("\nDifferences between base and partition in validation set:")
print("Number of different elements:", base_partition_diffs)
n_flips = base_partition_diffs

### Apply the PROMIS methods

In [ ]:
wlimit=300#3600
max_pr_shift=0.1
promis_methods = [
    "promis_app",
    "promis_opt",
]

for method in promis_methods:
    fair_model = SpatialOptimFairnessModel(method)
    fair_model.multi_fit(
        points_per_region=pts_per_region_val,
        n_flips_start = n_flips_start,
        step=n_flips_start,
        n_flips=n_flips,
        y_pred=y_val_pred_base,
        y_true=y_val if fair_notion == "equal_opportunity" else None,
        y_pred_probs=y_val_base_pred_proba,
        wlimit=wlimit,
        fair_notion=fair_notion,
        overlap=overlap,
        no_of_threads=0,
        verbose=1,
        max_pr_shift=max_pr_shift
    )

    if method == "promis_opt":
        model_save_file = f"{sp_opt_models_dir}/{method}_wlimit_{wlimit}.pkl"
    else:
        model_save_file = f"{sp_opt_models_dir}/{method}.pkl"

    fair_model.save_model(model_save_file)